In [1]:
from dolfin import *
import numpy as np, sklearn.metrics.pairwise as sp

In [2]:
# 应变voigt法表示
def strain3voigt(tensorStrain):
    return as_vector([tensorStrain[0,0],
                      tensorStrain[1,1],
                      tensorStrain[2,2],
                      2*tensorStrain[1,2],
                      2*tensorStrain[2,0],
                      2*tensorStrain[0,1]])

# voigt应力tensor表示
def stress3tensor(voigtVector):
    return as_tensor([[voigtVector[0], voigtVector[5], voigtVector[4]],
                      [voigtVector[5], voigtVector[1], voigtVector[3]],
                      [voigtVector[4], voigtVector[3], voigtVector[2]]])

# 材料应变
def strain(u):
    return 1.0/2.0*(grad(u) + grad(u).T)

# 转轴公式
def RotateX(alpha):
    RtX = as_tensor([[1, 0, 0, 0, 0, 0],
                    [0, np.cos(alpha)**2, np.sin(alpha)**2, -2*np.sin(alpha)*np.cos(alpha), 0, 0],
                    [0, np.sin(alpha)**2, np.cos(alpha)**2, 2*np.sin(alpha)*np.cos(alpha), 0, 0],
                    [0, -np.sin(alpha)*np.cos(alpha), np.sin(alpha)*np.cos(alpha), np.cos(alpha)**2-np.sin(alpha)**2, 0, 0],
                    [0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 0, 1]]) 
    return RtX

def RotateY(beta):
    RtY = as_tensor([[np.cos(beta)**2, 0, np.sin(beta)**2, 0, -2*np.sin(beta)*np.cos(beta), 0],
                    [0, 1, 0, 0, 0, 0],
                    [np.sin(beta)**2, 0, np.cos(beta)**2, 0, 2*np.sin(beta)*np.cos(beta), 0],
                    [0, 0, 0, 1, 0, 0],
                    [-np.sin(beta)*np.cos(beta), np.sin(beta)*np.cos(beta), 0, 0, np.cos(beta)**2-np.sin(beta)**2, 0],
                    [0, 0, 0, 0, 0, 1]]) 
    return RtY

def RotateZ(gama):
    RtZ = as_tensor([[np.cos(gama)**2, np.sin(gama)**2, 0, 0, 0, -2*np.sin(gama)*np.cos(gama)],
                    [np.sin(gama)**2, np.cos(gama)**2, 0, 0, 0, 2*np.sin(gama)*np.cos(gama)],
                    [0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 0, 1, 0],
                    [-np.sin(gama)*np.cos(gama), np.sin(gama)*np.cos(gama), 0, 0, 0, np.cos(gama)**2-np.sin(gama)**2]]) 
    return RtZ

mE1 = 114e9
mE2 = 8.61e9
mE3 = 8.61e9
mMu12 = 0.3
mMu13 = 0.3
mMu23 = 0.45
mG12 = 4.16e9
mG13 = 4.16e9
mG23 = 3.0e9

Cons = (1-mMu12*mMu12-mMu23*mMu23-mMu13*mMu13-2*mMu12*mMu23*mMu13)/(mE1*mE2*mE3)
mC11 = (1-mMu23*mMu23)/(mE2*mE3*Cons)
mC22 = (1-mMu13*mMu13)/(mE1*mE3*Cons)
mC33 = (1-mMu12*mMu12)/(mE1*mE2*Cons)
mC12 = mC21 = (mMu12+mMu23*mMu13)/(mE1*mE3*Cons)
mC13 = mC31 = (mMu13+mMu12*mMu23)/(mE1*mE2*Cons)
mC23 = mC32 = (mMu23+mMu12*mMu13)/(mE1*mE2*Cons)
mC44 = mG23
mC55 = mG13
mC66 = mG12

# 正交各向异性材料弹性矩阵
Cijkl = Cons*as_tensor([[mC11, mC12, mC13,    0,    0,    0],
                        [mC21, mC22, mC23,    0,    0,    0],
                        [mC31, mC32, mC33,    0,    0,    0],
                        [   0,    0,    0, mC44,    0,    0],
                        [   0,    0,    0,    0, mC55,    0],
                        [   0,    0,    0,    0,    0, mC66]])

# 材料应变
def strain(u):
    return 1.0/2.0*(grad(u) + grad(u).T)

# 材料应力
def stress(u, Rx, Ry, Rz):
    Rotate = RotateX(Rx)*RotateY(Ry)*RotateZ(Rz)
    sts = dot(dot(Rotate, dot(Cijkl, strain3voigt(strain(u)))), Rotate.T)
    return stress3tensor(sts)



In [3]:
# A 55 LINE TOPOLOGY OPTIMIZATION CODE ---------------------------------
def main(nelx, nely, nelz, volfrac, penal, rmin):
    # 本构方程，有限元准备
    sigma = lambda _u: 2.0 * mu * sym(grad(_u)) + lmbda * tr(sym(grad(_u))) * Identity(len(_u))
    psi = lambda _u: lmbda / 2 * (tr(sym(grad(_u))) ** 2) + mu * tr(sym(grad(_u)) * sym(grad(_u)))
    xdmf = XDMFFile("output/density.xdmf")
    mu, lmbda = Constant(0.3846), Constant(0.5769)
    # PREPARE FINITE ELEMENT ANALYSIS ----------------------------------
    mesh = BoxMesh(Point(0,0,0), Point(nelx,nely,nelz), nelx, nely, nelz)
    # mesh = RectangleMesh(Point(0, 0), Point(nelx, nely), nelx, nely, "right/left")
    U = VectorFunctionSpace(mesh, "P", 1)
    D = FunctionSpace(mesh, "DG", 0)
    u, v = TrialFunction(U), TestFunction(U)
    u_sol, density, density_old, density_new = Function(U), Function(D, name="density"), Function(D), Function(D)
    density.vector()[:] = volfrac
    V0 = assemble(1 * TestFunction(D) * dx)
    # DEFINE SUPPORT ---------------------------------------------------
    support = CompiledSubDomain("near(x[1], 0.0, tol) && on_boundary", tol=1e-14)
    bcs = [DirichletBC(U, Constant((0.0, 0.0, 0.0)), support)]
    # DEFINE LOAD ------------------------------------------------------
    load_marker = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
    CompiledSubDomain("x[1]==l && x[0]<=7 && x[0]>=3 && x[2]<=1", l=nely).mark(load_marker, 1)
    ds = Measure("ds")(subdomain_data=load_marker)
    F = dot(v, Constant((0.0, 0.0, -1000.0))) * ds(1)
    # SET UP THE VARIATIONAL PROBLEM AND SOLVER ------------------------
    # K = inner(density ** penal * sigma(u), grad(v)) * dx
    K = inner(stress(u, 0, 0, 0), grad(v)) * dx
    solver = LinearVariationalSolver(LinearVariationalProblem(K, F, u_sol, bcs))
    # PREPARE DISTANCE MATRICES FOR FILTER -----------------------------
    midpoint = [cell.midpoint().array()[:] for cell in cells(mesh)]
    distance_mat = np.maximum(rmin - sp.euclidean_distances(midpoint, midpoint), 0)
    distance_sum = distance_mat.sum(1)
    # START ITERATION --------------------------------------------------
    loop, change = 0, 1
    while change > 0.01 and loop < 100:
        loop = loop + 1
        density_old.assign(density)
        # FE-ANALYSIS --------------------------------------------------
        solver.solve()
        # OBJECTIVE FUNCTION AND SENSITIVITY ANALYSIS ------------------
        objective = density ** penal * psi(u_sol)
        sensitivity = project(-diff(objective, density), D).vector()[:]
        # FILTERING/MODIFICATION OF SENSITIVITIES ----------------------
        sensitivity = np.divide(distance_mat @ np.multiply(density.vector()[:], sensitivity), np.multiply(density.vector()[:], distance_sum))
        # DESIGN UPDATE BY THE OPTIMALITY CRITERIA METHOD --------------
        l1, l2, move = 0, 100000, 0.2
        while l2 - l1 > 1e-4:
            l_mid = 0.5 * (l2 + l1)
            density_new.vector()[:] = np.maximum(0.001, np.maximum(density.vector()[:] - move, np.minimum(1.0, np.minimum(density.vector()[:] + move, density.vector()[:] * np.sqrt(-sensitivity / V0 / l_mid)))))
            current_vol = assemble(density_new * dx)
            l1, l2 = (l_mid, l2) if current_vol > volfrac * V0.sum() else (l1, l_mid)
        # PRINT RESULTS ------------------------------------------------
        change = max(density_new.vector()[:] - density_old.vector()[:])
        print("it.: {0} , obj.: {1:.3f} Vol.: {2:.3f}, ch.: {3:.3f}".format(loop, project(objective, D).vector().sum(), current_vol / V0.sum(), change))
        density.assign(density_new)
        xdmf.write(density, loop)

In [4]:
set_log_level(50)
main(nelx=10, nely=50, nelz=10, volfrac=0.5, penal=3.0, rmin=2.0)

it.: 1 , obj.: 916532166595386132391527923696938059314334091182080.000 Vol.: 0.700, ch.: 0.200
it.: 2 , obj.: 2514964265137749656061260566909611468211226178224128.000 Vol.: 0.900, ch.: 0.200
it.: 3 , obj.: 5345215595584330558119088339580388711073403467988992.000 Vol.: 1.000, ch.: 0.100
it.: 4 , obj.: 7332257332763089059132223389575504474514672729456640.000 Vol.: 1.000, ch.: 0.000
